In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

# Load the dataset from file
data_path = '/content/drive/MyDrive/TAS/Task 2/dialogs.txt'
questions = []
answers = []

with open(data_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines:
        line = line.strip().split('\t')
        questions.append(line[0])
        answers.append(line[1])

# Preprocess the data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(questions + answers)

num_encoder_tokens = len(tokenizer.word_index) + 1

encoder_input_data = tokenizer.texts_to_sequences(questions)
encoder_input_data = pad_sequences(encoder_input_data)

decoder_input_data = tokenizer.texts_to_sequences(answers)
decoder_input_data = pad_sequences(decoder_input_data)

decoder_target_data = np.zeros_like(decoder_input_data)
decoder_target_data[:, 1:] = decoder_input_data[:, :-1]



In [ ]:
# Model Architecture
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_encoder_tokens, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_encoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Compile and train the model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=32, epochs=20)

Epoch 1/20
117/117 [==============================] - 43s 270ms/step - loss: 2.5854
Epoch 2/20
117/117 [==============================] - 32s 270ms/step - loss: 1.7292
Epoch 3/20
117/117 [==============================] - 33s 282ms/step - loss: 1.6453
Epoch 4/20
117/117 [==============================] - 32s 271ms/step - loss: 1.5354
Epoch 5/20
117/117 [==============================] - 32s 271ms/step - loss: 1.3690
Epoch 6/20
117/117 [==============================] - 31s 269ms/step - loss: 1.1891
Epoch 7/20
117/117 [==============================] - 33s 281ms/step - loss: 1.0416
Epoch 8/20
117/117 [==============================] - 32s 274ms/step - loss: 0.9186
Epoch 9/20
117/117 [==============================] - 32s 272ms/step - loss: 0.8074
Epoch 10/20
117/117 [==============================] - 32s 274ms/step - loss: 0.7055
Epoch 11/20
117/117 [==============================] - 33s 286ms/step - loss: 0.6147
Epoch 12/20
117/117 [==============================] - 32s 272ms/step - lo

In [ ]:
# Load and preprocess your dataset
data_path = '/content/drive/MyDrive/TAS/Task 2/dialogs.txt'  # Change to the path of your dataset file
questions = []
answers = []

with open(data_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines:
        line = line.strip().split('\t')
        questions.append(line[0])
        answers.append(line[1])

# Calculate the maximum sequence length
max_sequence_length = max(max(len(question.split()), len(answer.split())) for question, answer in zip(questions, answers))

print("Maximum Sequence Length:", max_sequence_length)


Maximum Sequence Length: 19


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

# Save the trained model
model.save('/content/drive/MyDrive/TAS/Task 2/chatbot_model.h5')

# Save the tokenizer
with open('tokenizer.pkl', 'wb') as token_file:
    pickle.dump(tokenizer, token_file)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# Load the tokenizer (Load or prepare tokenizer same as used during training)
# Load the tokenizer used during training
tokenizer_path = '/content/tokenizer.pkl'  # Path to the tokenizer file
with open(tokenizer_path, 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)


# Load the trained model
model_path = '/content/drive/MyDrive/TAS/Task 2/chatbot_model.h5'
model = load_model(model_path)

# Define the maximum sequence length (set to the maximum length of sequences in your dataset)
max_sequence_length = 19

def evaluate(sentence):
    try:
        # Preprocess the input sentence
        sentence = sentence.lower()  # Convert to lowercase
        input_sequence = tokenizer.texts_to_sequences([sentence])
        input_sequence = pad_sequences(input_sequence, maxlen=max_sequence_length, padding='post')
        input_tensor = tf.convert_to_tensor(input_sequence)

        # Create a dummy decoder input
        dummy_decoder_input = np.zeros((input_tensor.shape[0], max_sequence_length))

        # Generate a response
        predicted_sequence = model.predict([input_tensor, dummy_decoder_input])
        predicted_ids = np.argmax(predicted_sequence, axis=-1)
        response = ' '.join(tokenizer.index_word[idx] for idx in predicted_ids[0] if idx != 0)

        predicted_sequence = model.predict([input_tensor, dummy_decoder_input])
        predicted_ids = np.argmax(predicted_sequence, axis=-1)
        response = ' '.join(tokenizer.index_word[idx] for idx in predicted_ids[0] if idx != 0)

        predicted_ids = np.argmax(predicted_sequence, axis=-1)
        print("Predicted IDs:", predicted_ids)  # Debugging line
        response = ' '.join(tokenizer.index_word[idx] for idx in predicted_ids[0] if idx != 0)

        return response
    except Exception as e:
        print("Error during prediction:", e)
        return "Sorry, I couldn't generate a response."

# Interact with the chatbot
while True:
    user_input = input("You: ")

    if user_input.lower() == 'quit':
        print("Chatbot: Goodbye!")
        break

    response = evaluate(user_input)
    print("Chatbot:", response)


In [ ]:
import numpy as np
import string
from nltk.corpus import stopwords
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/TAS/Task 2/dialogs.txt',sep='\t')

In [ ]:
a = pd.Series(df.columns)
df

hi, how are you doing?  \
0                         i'm fine. how about yourself?   
1                   i'm pretty good. thanks for asking.   
2                     no problem. so how have you been?   
3                      i've been great. what about you?   
4              i've been good. i'm in school right now.   
...                                                 ...   
3719    that's a good question. maybe it's not old age.   
3720                              are you right-handed?   
3721                                  yes. all my life.   
3722  you're wearing out your right hand. stop using...   
3723        but i do all my writing with my right hand.   

                          i'm fine. how about yourself?  
0                   i'm pretty good. thanks for asking.  
1                     no problem. so how have you been?  
2                      i've been great. what about you?  
3              i've been good. i'm in school right now.  
4                             what school do you go to?  
...                                                 ...  
3719                              are you right-handed?  
3720                                  yes. all my life.  
3721  you're wearing out your right hand. stop using...  
3722        but i do all my writing with my right hand.  
3723  start typing instead. that way your left hand ...  

[3724 rows x 2 columns]

In [ ]:
a = a.rename({0: df.columns[0],1: df.columns[1]})

In [ ]:
b = {'Questions':'Hi','Answers':'hello'}

c = {'Questions':'Hello','Answers':'hi'}

d= {'Questions':'how are you','Answers':"i'm fine. how about yourself?"}

e= {'Questions':'how are you doing','Answers':"i'm fine. how about yourself?"}

In [ ]:
df = df.append(a,ignore_index=True)

<ipython-input-65-8a35d942b5da>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(a,ignore_index=True)


In [ ]:
df.columns=['Questions','Answers']

In [ ]:
df = df.append([b,c,d,e],ignore_index=True)
df

<ipython-input-67-a52826071be3>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append([b,c,d,e],ignore_index=True)


Questions  \
0                i'm fine. how about yourself?   
1          i'm pretty good. thanks for asking.   
2            no problem. so how have you been?   
3             i've been great. what about you?   
4     i've been good. i'm in school right now.   
...                                        ...   
3724                    hi, how are you doing?   
3725                                        Hi   
3726                                     Hello   
3727                               how are you   
3728                         how are you doing   

                                       Answers  
0          i'm pretty good. thanks for asking.  
1            no problem. so how have you been?  
2             i've been great. what about you?  
3     i've been good. i'm in school right now.  
4                    what school do you go to?  
...                                        ...  
3724             i'm fine. how about yourself?  
3725                                     hello  
3726                                        hi  
3727             i'm fine. how about yourself?  
3728             i'm fine. how about yourself?  

[3729 rows x 2 columns]

In [ ]:
df = df.append(c,ignore_index=True)

df = df.append(d,ignore_index=True)

<ipython-input-68-d820f5d172f0>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(c,ignore_index=True)
<ipython-input-68-d820f5d172f0>:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(d,ignore_index=True)


In [ ]:
df

Questions  \
0                i'm fine. how about yourself?   
1          i'm pretty good. thanks for asking.   
2            no problem. so how have you been?   
3             i've been great. what about you?   
4     i've been good. i'm in school right now.   
...                                        ...   
3726                                     Hello   
3727                               how are you   
3728                         how are you doing   
3729                                     Hello   
3730                               how are you   

                                       Answers  
0          i'm pretty good. thanks for asking.  
1            no problem. so how have you been?  
2             i've been great. what about you?  
3     i've been good. i'm in school right now.  
4                    what school do you go to?  
...                                        ...  
3726                                        hi  
3727             i'm fine. how about yourself?  
3728             i'm fine. how about yourself?  
3729                                        hi  
3730             i'm fine. how about yourself?  

[3731 rows x 2 columns]

In [ ]:
def cleaner(x):
    return [a for a in (''.join([a for a in x if a not in string.punctuation])).lower().split()]

In [ ]:
Pipe = Pipeline([
    ('bow',CountVectorizer(analyzer=cleaner)),
    ('tfidf',TfidfTransformer()),
    ('classifier',DecisionTreeClassifier())
])

In [ ]:
Pipe.fit(df['Questions'],df['Answers'])

In [ ]:
# Assuming you've already loaded the model pipeline as 'Pipe'

while True:
    user_input = input("You: ")

    if user_input.lower() == 'quit':
        print("Chatbot: Goodbye!")
        break

    response = Pipe.predict([user_input])[0]
    print("Chatbot:", response)

You: hi
Chatbot: hello
You: how are you
Chatbot: i'm fine. how about yourself?
You: i'm fine
Chatbot: so am i.
You: how are you doing
Chatbot: i'm fine. how about yourself?
You: quit
Chatbot: Goodbye!
